In [213]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [220]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [183]:
# Read file and parse dates if any present
df = pd.read_csv('telecom_churn_data.csv', parse_dates = True)

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [185]:
df.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


In [186]:
# Check the rows and columns in the dataframe
df.shape

(99999, 226)

In [187]:
# Check count of unique mobile numbers
len(df['mobile_number'].unique())

99999

In [188]:
# Since all the mobile numbers are unique, let's set the "mobile number" as the INDEX
df = df.set_index('mobile_number')

In [189]:
# Check for columns with all values as 'na'
print('The number of columns with all values missing/NA are:\r\n',len([i for i in df.columns if df[i].isna().all()]))

The number of columns with all values missing/NA are:
 0


In [190]:
#Check if there are rows with all values as 'na'
print('The number of rows with all values missing/NA are:\r\n',len(df.index[df.isna().all(1)]))

The number of rows with all values missing/NA are:
 0


In [191]:
# Find columns with greater than 50% of values null
null_columns=df.columns[df.isnull().any()]
for i in null_columns:
    if(df[i].isnull().sum()*100/df.shape[0] >50):
        print(i,":",df[i].isnull().sum())

date_of_last_rech_data_6 : 74846
date_of_last_rech_data_7 : 74428
date_of_last_rech_data_8 : 73660
date_of_last_rech_data_9 : 74077
total_rech_data_6 : 74846
total_rech_data_7 : 74428
total_rech_data_8 : 73660
total_rech_data_9 : 74077
max_rech_data_6 : 74846
max_rech_data_7 : 74428
max_rech_data_8 : 73660
max_rech_data_9 : 74077
count_rech_2g_6 : 74846
count_rech_2g_7 : 74428
count_rech_2g_8 : 73660
count_rech_2g_9 : 74077
count_rech_3g_6 : 74846
count_rech_3g_7 : 74428
count_rech_3g_8 : 73660
count_rech_3g_9 : 74077
av_rech_amt_data_6 : 74846
av_rech_amt_data_7 : 74428
av_rech_amt_data_8 : 73660
av_rech_amt_data_9 : 74077
arpu_3g_6 : 74846
arpu_3g_7 : 74428
arpu_3g_8 : 73660
arpu_3g_9 : 74077
arpu_2g_6 : 74846
arpu_2g_7 : 74428
arpu_2g_8 : 73660
arpu_2g_9 : 74077
night_pck_user_6 : 74846
night_pck_user_7 : 74428
night_pck_user_8 : 73660
night_pck_user_9 : 74077
fb_user_6 : 74846
fb_user_7 : 74428
fb_user_8 : 73660
fb_user_9 : 74077


In [192]:
df.filter(regex = 'total_rech_data_').min()

total_rech_data_6    1.0
total_rech_data_7    1.0
total_rech_data_8    1.0
total_rech_data_9    1.0
dtype: float64

In [193]:
print("No. of rows where data recharge is not equal to 2G and 3G recharge in month 6:",df[(df['total_rech_data_6'] != df['count_rech_2g_6'] + df['count_rech_3g_6'])].shape[0])
print("No. of rows where data recharge is not equal to 2G and 3G recharge in month 7:",df[(df['total_rech_data_7'] != df['count_rech_2g_7'] + df['count_rech_3g_7'])].shape[0])
print("No. of rows where data recharge is not equal to 2G and 3G recharge in month 8:",df[(df['total_rech_data_8'] != df['count_rech_2g_8'] + df['count_rech_3g_8'])].shape[0])
print("No. of rows where data recharge is not equal to 2G and 3G recharge in month 9:",df[(df['total_rech_data_9'] != df['count_rech_2g_9'] + df['count_rech_3g_9'])].shape[0])

No. of rows where data recharge is not equal to 2G and 3G recharge in month 6: 74846
No. of rows where data recharge is not equal to 2G and 3G recharge in month 7: 74428
No. of rows where data recharge is not equal to 2G and 3G recharge in month 8: 73660
No. of rows where data recharge is not equal to 2G and 3G recharge in month 9: 74077


#### From the above, it can be observed that "total_rech_data_X" which represents "number of recharges" in the month has a minimum value 1. So if a customer (high-value or not) does not recharge for data in a month, the value is missing and not 0. This explains the same count of missing values that appear across other data-related columns above.

In [194]:
print("Assumption 1: If the data recharge values are missing, the customers have not recharged.",
      "So, impute 0 (zero) where the recharge values (total_rech, max_rech, count_rech, av_rech, arpu) are missing.")

Assumption 1: If the data recharge values are missing, the customers have not recharged. So, impute 0 (zero) where the recharge values (total_rech, max_rech, count_rech, av_rech, arpu) are missing.


In [195]:
for i in df.filter(regex="(total_rech_data|max_rech_data|count_rech_.g|av_rech_amt_data|arpu_.g)").columns:
    df[i] = df[i].fillna(value = 0)

In [196]:
print("Assumption 2: The packs NIGHT and FB_USER are related to data packs and so they will be imputed with 0.")

Assumption 2: The packs NIGHT and FB_USER are related to data packs and so they will be imputed with 0.


In [209]:
for i in df.filter(regex="(night_pck_user|fb_user)").columns:
    df[i] = df[i].fillna(value = 0)

In [198]:
#print("Assumption 3: If minutes of usage values are missing, assume that the customers have not made calls.",
#      "So, impute 0 (zero) where the minutes of usage (mou, ic,og) are missing.")

In [199]:
#for i in df.filter(regex="(mou|og|ic)").columns:
#    df[i] = df[i].fillna(value = 0)

In [200]:
# Check the number of missing/NA values in each column
null_columns=df.columns[df.isnull().any()]
print('Following is the count of null values in each column:')
print(df[null_columns].isnull().sum())

Following is the count of null values in each column:
loc_og_t2o_mou               1018
std_og_t2o_mou               1018
loc_ic_t2o_mou               1018
last_date_of_month_7          601
last_date_of_month_8         1100
last_date_of_month_9         1659
onnet_mou_6                  3937
onnet_mou_7                  3859
onnet_mou_8                  5378
onnet_mou_9                  7745
offnet_mou_6                 3937
offnet_mou_7                 3859
offnet_mou_8                 5378
offnet_mou_9                 7745
roam_ic_mou_6                3937
roam_ic_mou_7                3859
roam_ic_mou_8                5378
roam_ic_mou_9                7745
roam_og_mou_6                3937
roam_og_mou_7                3859
roam_og_mou_8                5378
roam_og_mou_9                7745
loc_og_t2t_mou_6             3937
loc_og_t2t_mou_7             3859
loc_og_t2t_mou_8             5378
loc_og_t2t_mou_9             7745
loc_og_t2m_mou_6             3937
loc_og_t2m_mou_7            

In [201]:
df.shape

(99999, 225)

#### Shortlist high-value customers
##### "Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase)."

In [202]:
df['total'] = df[['total_rech_amt_6','total_rech_amt_7']].sum(axis=1)
threshold = float(df['total'].quantile([0.70]))
df = df[df['total'] > threshold]
df = df.drop('total', axis=1)

In [203]:
df.shape

(29979, 225)

#### Identify churn and non-churn customers using the criteria:
##### "Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes to use to tag churners are: total_ic_mou_9, total_og_mou_9, vol_2g_mb_9, vol_3g_mb_9"

In [204]:
# Label churn and non-churn customers
df['churn'] = np.where(
            (
                (df['total_ic_mou_9'] == 0.0) | 
                (df['total_og_mou_9'] == 0.0)
            ) & 
            (
                (df['vol_2g_mb_9'] == 0.0) | 
                (df['vol_3g_mb_9'] == 0.0)
            ),1,0
        )

In [205]:
# Look at the distribution of churn and non-churn customers
df['churn'].value_counts()

0    26936
1     3043
Name: churn, dtype: int64

In [206]:
# Remove columns with '9'
df = df.drop(df.filter(regex = '9').columns, axis=1)

In [207]:
df.shape

(29979, 172)

In [219]:
df.describe()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,churn
count,29979.0,29865.0,29865.0,29865.0,29979.000000,29979.000000,29979.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.00000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.00000,29041.000000,29663.0,29676.0,29041.0,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29979.000000,29979.000000,29979.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29663.0,29676.0,29041.0,29663.000000,29676.000000,29041.000000,29979.000000,29979.000000,29979.000000,29663.000000,29676.000000,29041.00000,29663.000000,29676.000000,29041.000000,29663.000000,29676.000000,29041.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.000000,29979.